In [1]:
from model import run_inference
from ultralytics import YOLO
import onnxruntime as ort
import numpy as np
import cv2
import json

In [3]:
model_path = "yolo11s.onnx"
onnx_model = ort.InferenceSession(model_path)

with open("class_names.json", 'r') as file:
    class_names = json.load(file)
    class_names = {int(k) : v for k, v in class_names.items()}

image = cv2.imread("../groceries.png")
cv2.imshow("Original", image)
_ = cv2.waitKey(0)

In [4]:
img_w, img_h = image.shape[1], image.shape[0]
print(f"img width: {img_w} \nimg height: {img_h}")

img width: 900 
img height: 643


In [5]:
img = cv2.resize(image, (640, 640))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.transpose(2, 0, 1)
img = img.reshape(1, 3, 640, 640)
img = img / 255.0
img = img.astype(np.float32)
img.shape

(1, 3, 640, 640)

In [6]:
outputs = onnx_model.run(None, {"images" : img})
results = outputs[0]
results = results.transpose()
results.shape

(8400, 84, 1)

In [7]:
A = []
for detection in results:
    class_id = detection[4:].argmax()
    confidence_score = detection[4:].max()

    new_detection = np.append(detection[:4], [class_id, confidence_score])
    A.append(new_detection)
A = np.array(A)

predictions = np.array([detection for detection in A if detection[-1] > 0.2])
predictions.shape

(109, 6)

In [8]:
names = []
for pred in predictions:
    class_id = int(pred[-2])
    names.append((class_names[class_id], pred[-1]))

sorted_results = sorted(names, key=lambda x: x[1], reverse=True)
top_results = set([x[0] for x in sorted_results])
print(top_results)


{'banana', 'orange', 'apple', 'broccoli'}
